In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import os
%cd / 

/


In [3]:
def load_dataset(paths):
    data = []
    label = []
    value = 0
    for path in paths:
        print(path)
        if path.split('/')[-1] == 'pos':
            value = 1
        else:
            value = 0
        for son in os.listdir(path):
            new_path = os.path.join(path,son)
            with open(new_path,'r') as f:
                lines = f.readlines()
                for line in lines:
                    data.append(line)
                    label.append(value)
    return data,label
train_data,train_label = load_dataset(['/kaggle/input/acllmdb/aclImdb/train/pos','/kaggle/input/acllmdb/aclImdb/train/neg'])#'/kaggle/input/acllmdb/aclImdb/train/neg'
test_data,test_label = load_dataset(['/kaggle/input/acllmdb/aclImdb/test/pos','/kaggle/input/acllmdb/aclImdb/test/neg'])#,'/kaggle/input/acllmdb/aclImdb/test/neg'


/kaggle/input/acllmdb/aclImdb/train/pos
/kaggle/input/acllmdb/aclImdb/train/neg
/kaggle/input/acllmdb/aclImdb/test/pos
/kaggle/input/acllmdb/aclImdb/test/neg


In [4]:
# !pip install transformers
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bertmodel = BertModel.from_pretrained('bert-base-uncased')
# ret = tokenizer.tokenize('what are you doing now')
# input_ids = tokenizer.convert_tokens_to_ids(ret)
# inputs = tokenizer.encode('Hello, my dog is cute',max_length = 3)
# inputssss = tokenizer('Hello, my dog is cute',max_length = 512)
# print(inputssss)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
# for name,para in model.named_parameters():
#     print(name,':',para)
# print(model.config.hidden_size)

In [6]:
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
class myModel(torch.nn.Module):
    def __init__(self,bertmodel,dropout=None):
        super(myModel,self).__init__()
        self.bertmodel = bertmodel
        self.dropout = nn.Dropout(dropout if dropout is not None else 0.1)
        # 语义匹配任务: 相似、不相似 2 分类任务
        self.classifier = nn.Linear(self.bertmodel.config.hidden_size, 2)
    def forward(self,input_ids,token_type_ids,attention_mask):
        outputs= self.bertmodel(input_ids,token_type_ids = token_type_ids, attention_mask=attention_mask)#
        outputs = outputs.pooler_output
        outputs = self.dropout(outputs)
        # 基于文本对的语义表示向量进行 2 分类任务
        logits = self.classifier(outputs)
        return logits
model = myModel(bertmodel)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
model = model.to(device)

cuda


In [7]:
# import numpy as np
# def convert_examples_to_feature(examples,tokenizer =tokenizer,max_length = 512):
#     data = []
#     for text in examples:
#         encoder_inputs = tokenizer(text,max_length)
#         input_ids = encoder_inputs['input_ids']
#         data.append(input_ids)
#     return data

# train_data = convert_examples_to_feature(train_data)
# test_data = convert_examples_to_feature(test_data)

In [8]:
# print(len(train_data))
# print(train_data[0])

In [9]:
from torch.utils.data import Dataset
class myDataset(Dataset):
    def __init__(self,data,label):
        super(myDataset).__init__()
        self.input_ids = []
        self.token_type_ids = []
        self.attention_mask = []
        self.label = label
        for text in data:
            encoder_inputs = tokenizer(text,max_length = 128)
            self.input_ids.append(encoder_inputs['input_ids'])
            self.token_type_ids.append(encoder_inputs['token_type_ids'])
            self.attention_mask.append(encoder_inputs['attention_mask'])
    def __getitem__(self,idx):
        input_ids = self.input_ids[idx]
        token_type_ids = self.token_type_ids[idx]
        attention_mask = self.attention_mask[idx]
        label = self.label[idx]
        return torch.tensor(input_ids, dtype=torch.long),torch.tensor(token_type_ids, dtype=torch.long),torch.tensor(attention_mask, dtype=torch.long),torch.tensor(label, dtype=torch.long)
    def __len__(self):
        return len(self.input_ids)
train_dataset = myDataset(train_data,train_label)
test_dataset = myDataset(test_data,test_label)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [10]:
import numpy as np
def my_collate_fn(data):
    new_input_ids = []
    new_token_type_ids = []
    new_attention_mask = []
    new_y = []
    max_length = 0
    for input_ids,token_type_ids,attention_mask,label in data:
        token_length = len(input_ids)
        max_length = max(max_length,token_length)
    if max_length > 512:
        max_length = 512
    for input_ids,token_type_ids,attention_mask,label in data:
        token_length = len(input_ids)
        input_ids = np.array(input_ids)
        token_type_ids = np.array(token_type_ids)
        attention_mask = np.array(attention_mask)
#         label = np.array(label)   因为这里取出的label就是一个tensor（1），其里面是一个int整数，如果转成np.array，后面再转tensor就会报no len（）错，错误描述可以看下一个cell
        if token_length < max_length:
            input_ids = np.concatenate([input_ids[:-1],[tokenizer.pad_token_id] * (max_length - token_length) + [tokenizer.sep_token_id]],axis = 0)
            token_type_ids = np.array(token_type_ids)
            token_type_ids = np.concatenate([token_type_ids,[0] * (max_length - token_length)],axis = 0)
            attention_mask = np.array(attention_mask)
            attention_mask = np.concatenate([attention_mask,[0]* (max_length - token_length)],axis = 0)
        new_input_ids.append(input_ids)
        new_token_type_ids.append(token_type_ids)
        new_attention_mask.append(attention_mask)
        new_y.append(label)
    new_input_ids = torch.tensor(new_input_ids,dtype=torch.long)
    new_token_type_ids = torch.tensor(new_token_type_ids,dtype=torch.long)
    new_attention_mask = torch.tensor(new_attention_mask,dtype=torch.long)
    new_y = torch.tensor(new_y,dtype=torch.long)
    return new_input_ids, new_token_type_ids,new_attention_mask,new_y

In [11]:
# a = torch.tensor(1)
# b = torch.tensor(2)
# c = np.array(a)
# d = np.array(b)
# e = [c,d]
# print(torch.tensor(e))
# TypeError: len() of unsized object  会报这个错是因为 a中值是一个int型，其没有len（）函数，而在转tensor时，会调取其len（）,故报错，因此上面的label不能转成np.ndarray

In [12]:
from torch.utils.data import DataLoader
print(len(train_dataset))
train_dataset, valid_dataset = torch.utils.data.random_split(train_dataset, [20000, 5000])
train_data_loader = DataLoader(
                train_dataset,
                batch_size = 64,
                collate_fn = my_collate_fn,
                shuffle=True
)
valid_data_loader = DataLoader(
                valid_dataset,
                batch_size = 64,
                collate_fn = my_collate_fn,
                shuffle=True
)
test_data_loader = DataLoader(
                test_dataset,
                batch_size = 32,
                shuffle=True,
                collate_fn = my_collate_fn
)

25000


In [13]:
count = 0 
for batch in valid_dataset:
    if batch[-1] == 0:
        count += 1
print(count / 5000)

0.499


In [14]:
import time
import torch.nn as nn
from torch import optim
from torch.optim import lr_scheduler
criterion = nn.CrossEntropyLoss()
criterion = criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0001)
scheduler = lr_scheduler.MultiStepLR(optimizer,milestones=[20,80],gamma = 0.9)
epochs = 5
# !pip install torchmetrics
import torchmetrics as metrics
train_accuracy = metrics.Accuracy()
train_accuracy.to(device)
ckpt_dir = './'

In [15]:
torch.cuda.empty_cache()

In [16]:
import numpy as np
global_step = 0
tic_train = time.time()
model.train()
for epoch in range(1, epochs + 1):
    train_loss = 0
    for step, batch in enumerate(train_data_loader, start=1):
        optimizer.zero_grad()
        input_ids,token_type_ids,attention_mask,labels = batch
        input_ids = input_ids.to(device,dtype=torch.long)
        token_type_ids = token_type_ids.to(device,dtype=torch.long)
        attention_mask =attention_mask.to(device,dtype=torch.long)
        labels = labels.to(device,dtype=torch.long)
        # 喂数据给model
        probs = model(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)
        # 计算损失函数值
        loss = criterion(probs, labels)
        train_loss += loss
        loss.backward()
#         for name, parms in model.named_parameters():
# #             print(parms.data)
#             print('-->name:', name, '-->grad_requirs:', parms.requires_grad, '--weight', torch.mean(parms.data), ' -->grad_value:', torch.mean(parms.grad))
        optimizer.step()
        scheduler.step()
        # 预测分类概率值
        # 计算acc
        batch_acc = train_accuracy(probs, labels)
        global_step += 1
        if global_step % 30 == 0:
#             acc = train_accuracy.compute()
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, accu: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, batch_acc,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        
    print('epoch:%d,total_loss:%.5f,accu:%.5f' %(epoch,train_loss * epoch / global_step, train_accuracy.compute()))
    
    # 评估当前训练的模型  
    save_dir = os.path.join(ckpt_dir, "model_%d" % global_step)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir) 
    with torch.no_grad():
        torch.cuda.empty_cache()
        valid_accuracy = metrics.Accuracy()
        valid_accuracy.to(device)
        for  i, batch in enumerate(valid_data_loader):
            input_ids,token_type_ids,attention_mask,labels = batch
            input_ids = input_ids.to(device,dtype=torch.long)
            token_type_ids = token_type_ids.to(device,dtype=torch.long)
            attention_mask =attention_mask.to(device,dtype=torch.long)
            labels = labels.to(device,dtype=torch.long)
            probs = model(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)
            valid_accuracy(probs,labels)
        acc = valid_accuracy.compute()
        print('eval_acc: %.5f ' % acc)
        torch.save(model.state_dict(),os.path.join(save_dir,'model.pt'))

global step 30, epoch: 1, batch: 30, loss: 0.30070, accu: 0.90625, speed: 0.45 step/s
global step 60, epoch: 1, batch: 60, loss: 0.28492, accu: 0.90625, speed: 0.46 step/s
global step 90, epoch: 1, batch: 90, loss: 0.34072, accu: 0.82812, speed: 0.46 step/s
global step 120, epoch: 1, batch: 120, loss: 0.31917, accu: 0.89062, speed: 0.46 step/s
global step 150, epoch: 1, batch: 150, loss: 0.22120, accu: 0.90625, speed: 0.46 step/s
global step 180, epoch: 1, batch: 180, loss: 0.33422, accu: 0.85938, speed: 0.46 step/s
global step 210, epoch: 1, batch: 210, loss: 0.30382, accu: 0.85938, speed: 0.46 step/s
global step 240, epoch: 1, batch: 240, loss: 0.24825, accu: 0.89062, speed: 0.46 step/s
global step 270, epoch: 1, batch: 270, loss: 0.28896, accu: 0.82812, speed: 0.46 step/s
global step 300, epoch: 1, batch: 300, loss: 0.41404, accu: 0.85938, speed: 0.46 step/s
epoch:1,total_loss:0.35324,accu:0.83865
eval_acc: 0.86980 
global step 330, epoch: 2, batch: 17, loss: 0.27429, accu: 0.89062,

In [17]:
# save_dir = os.path.join(ckpt_dir, "model_%d" % 20)
# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)
# torch.save(model.state_dict(),os.path.join(save_dir,'model.pt'))

In [18]:
model.eval()
torch.cuda.empty_cache()
test_accuracy = metrics.Accuracy()
test_accuracy.to(device)
for step,batch in enumerate(test_data_loader):
    input_ids,token_type_ids,attention_mask,labels = batch
    input_ids = input_ids.to(device,dtype=torch.long)
    token_type_ids = token_type_ids.to(device,dtype=torch.long)
    attention_mask =attention_mask.to(device,dtype=torch.long)
    labels = labels.to(device,dtype=torch.long)
    probs = model(input_ids = input_ids, token_type_ids = token_type_ids, attention_mask = attention_mask)
    batch_acc = test_accuracy(probs, labels)
    if step != 0 and step % 100 == 0: 
        print("eval-> step:%d,batch_acc:%.5f" % (step,batch_acc))
print('!!!!eval-> total_acc:%.5f' % test_accuracy.compute())

eval-> step:100,batch_acc:1.00000
eval-> step:200,batch_acc:0.84375
eval-> step:300,batch_acc:0.84375
eval-> step:400,batch_acc:0.90625
eval-> step:500,batch_acc:0.84375
eval-> step:600,batch_acc:0.87500
eval-> step:700,batch_acc:0.93750
!!!!eval-> total_acc:0.87148


In [19]:
# from transformers import BertTokenizer, BertModel
# import torch

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model = BertModel.from_pretrained('bert-base-uncased')

# inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
# outputs = model(**inputs)
# last_hidden_states = outputs.pooler_output
# print(last_hidden_states)

In [20]:
# import torch
# import torch.nn as nn
# import numpy as np
# from torch.utils.data import Dataset
# from torch.utils.data import DataLoader
# class myDataset(Dataset):
#     def __init__(self,data,label):
#         super(myDataset).__init__()
#         self.input_ids = data
#         self.label = label
#     def __getitem__(self,idx):
#         return torch.tensor(self.input_ids[idx], dtype=torch.float),torch.tensor(self.label[idx], dtype=torch.float)
#     def __len__(self):
#         return len(self.input_ids)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# model = nn.Linear(1, 1)
# train_data = np.random.randn(100000).reshape(-1,1)
# train_label = np.array(train_data * 2 +1)
# test_data = np.random.randn(100000).reshape(-1,1)
# test_label = np.array(test_data * 2 +1)
# train_dataset = myDataset(train_data,train_label)
# test_dataset = myDataset(test_data,test_label)
# train_data_loader = DataLoader(
#                 train_dataset,
#                 batch_size = 16,
#                 shuffle=True
# )
# test_data_loader = DataLoader(
#                 test_dataset,
#                 batch_size = 16,
#                 shuffle=True
# )
# import torch.nn as nn
# from torch import optim 
# from torch.optim import lr_scheduler
# criterion = nn.MSELoss()
# criterion = criterion.to(device)
# model.to(device)
# optimizer = optim.Adam(model.parameters(), lr=0.01)
# scheduler = lr_scheduler.MultiStepLR(optimizer,milestones=[20,80],gamma = 0.9)
# epoch = 1
# for i in range(1,epoch+1):
#     for step,batch in enumerate(train_data_loader):
#         inputs,label = batch
#         inputs.to(device)
#         label.to(device)
#         model.to(device)
#         print(inputs[0])
#         y_hat = model(inputs)
#         loss = criterion(y_hat,label)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
#         scheduler.step()
# model.eval()
# print(model(torch.tensor([[0.1],[1.2]])))

In [21]:
# import torch
# import torchmetrics as metrics
# class RMSE(metrics.Metric):
#     def __init__(self):
#         self.add_state("sum_squared_errors",torch.tensor(0),dist_reduce_fx="sum")
#         self.add_atate("n_observations",torch.tensor(0),dist_reduce_fx = "sum")
#     def update(self,preds,target):
#         self.sum_squared_errors += torch.sum((pred - target) ** 2)
#         self.n_observations += preds.numel()
#     def compute(self):
#         return torch.sqrt(self.sum_squared_errors / self.n_observations)

In [22]:
# import torch
# import torchmetrics as metrics
# train_accuracy = metrics.Accuracy()
# # train_accuracy.to(device)
# a = np.random.randn(5,10)
# b = np.argmax(a,axis = -1)
# batch_acc = train_accuracy(torch.tensor(a),torch.tensor(b))
# b -= 1
# batch_acc = train_accuracy(torch.tensor(a),torch.tensor(b))
# acc=train_accuracy.compute()
# print(acc) 

In [23]:
import numpy as np
